# MIPT forum in maths and AI. Hackathon. Clever classifying. Written by Yaroslav Khripkov. In "collaboration" with other members of BadLab Team 

Перед началом, хотелось бы сказать о том, что не вошло в этом решение, но что мы обдумывали и пытались реализовать. 
> ### 50 оттенков предобработки

  Идеи были уже на этапе предобработки. Во-первых, использование готовых эмбеддингов, в частности были использованы компактные эмбеддинги navec. Помимо этого, данные были очищены от стоп-слов(что ухудшило качество классификации в данной задаче), очистка пунктуации, лемматизация слов. Последнее принесло небеольшой прирост качество, которым можно пожертвовать ради скорости работы и уменьшения вычислительной стоимости. После многих экспериментов мы решили, что в данной задаче знаки пунктуации, регистр символов и склонения несут большую смысловую нагрузку, но переписать свой "пайплайн" я не успевал, потому что исправлял модель в целом. 
  >### Война и  feature engineering

  Как я уже говорил, мы проанализировали качество моделей после разных методов предобработки, что позволило оценить вклад подобных мелочей в общее качество. К сожалению, писать feature extrator'ы мы начали слишком поздно, хотя даже простой счётчик вопросительных знаков и анализ количества заглавных букв дали прирост в пару тысячных. Кто знает, что могло бы получиться в итоге?

  >### Клуб борцов с дизбалансом классов

то борьба с несбалансированной выборкой. В ходе неё мы(Я) решили, что классические алгоритмы ***oversampling*** хорошо подойдут для генерации вопросов от экспертов. Однако оверсэплинг над векторами предложений после токенизации только сбил нас с пути тем, что встроенные методы показывали отличное качество на отложенной, но в то же время, сгерерированной выборке. Признайтесь, вам бы тоже хотелось иметь ROC-AUC и F1-Score больше 0.97. Расслабившись на время, мы взялись за голову вечером, когда осознали свою ошибку. После этого мы думали, как расширить выборку, чтобы избавиться от дисбаланса классов. Здесь могла бы подойти ***генерация подобных предложений с помощью реккурентых нейросетей***, но на это не было ни ресурсов, ни времени. После этого, мы принялись искать готовые эбмеддинги для предложений, а не слов, но возникли некоторые трудности с их имплементацией, что отняло время и потребовало нового решения. Хоть как то себя показало ***усреднение эмбедингов всех слов в предложении, а потом оверсэмплинг***. Данная техника позволила простой нейросети без LSTM и подобного получить ROC-AUC ~ 0.66. 
  Готовые модели для определения автора текста показали себя примерно так же. Далее были эксперименты с архитектурой сети и тестирования алгоритмов машинного обучения других семейств. Так мы пришли к CatBoost, который смог нас удивить своим качеством классификации "из коробки"
  >### Гарри Поттер и представление слов и предложений

Я пытался выбрать нужные методы токенизации/векторизации предложений/слов. Из-за огромного множества подходов и готовых решений проблемно попробовать всё. Первой идеей было использование предобученных эмбеддингов ELMo и RuBERT от DeepPavlov.ai, но из-за конфликтов версий внутри фреймворка, пришлось от этого отказаться. Далее был выбран ранее используемый мной navec, который и оказался в финальной версии. Обучаемые эмбеддинги не стоили прироста в скорости так, как много теряли в качестве. К сожалению, не хватило времени попробовать fasttext в данной задаче, ведь было интересно посмотреть, как он здесь отработает
>### Цветы для transfer learning

Я уже был готов самовыписаться из программистов, когда час поиска в google и яндексе не дал результатов. Очень много материалов вели к вышеупомянутому DeepPavlov.ai и его RuBERT и ELMo, встриванию которого мешало малое количество примеров и постоянные проблемы с google colab, который так и норовил "отвалиться"

>### Этюд в stacking'овых и enseble'вых тонах

У нас было 2 дня хакатона, 75 пачек печенья, 5 банок энергетиков, полпачки сигарет..
Естественно, большая часть времени ушла на очистку кода и его отладку. Некоторая - на research. Оставшегося категорически нехватало на отладку рабочей ансамблевой модели. Добавление всего лишь предсказания логистической регресии для предложения в выборку позволило увеличить метрику аж на одну десятую. Можно лишь представлять то, что бы выдавала модель, если бы к ней был добавлен еще и классификатор из KNearestNeighbours по эмбеддингам...

>### Завершение

Некоторые идеи были отвергнуты, многие остались нереализованными, а какие-то нашли своё место в нашем финальном решении. Возможно, я когда-нибудь вернусь к этому проекту, ведь проектов с NLP у меня было немного, а опыт нужен всегда. ***Было интересно работать над этой задачей, а еще интереснее, приятнее и полезнее Было бы по окончании форума и хакатона оказаться на стажировке)***




# Переходим от слов к делу. И к новым словам. Английским.

### Importing  and installing libraries. Unzip the data

In [54]:
import pandas as pd
import numpy as np
import pickle
import gensim
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
!pip install catboost
import catboost
from catboost import Pool, CatBoostClassifier
from sklearn import model_selection
!pip install natasha
!pip install slovnet
from string import punctuation
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)
from sklearn.metrics import roc_auc_score
from sklearn import model_selection
!unzip clever-mipt.zip




Archive:  clever-mipt.zip
replace Dockerfile? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


Read the data from .csv ans selected neccesary features

In [55]:
data=pd.read_csv('data.csv', sep=';', index_col=None)
ans=pd.read_csv('train.csv', sep=';', index_col=None)
data.drop('ID', axis=1)
ans.drop('ID', axis=1)
data['Question']=data['Question'].astype(str)
data_train=data['Question'][:30000]
data_test=data['Question'][30000:]

noise = list(punctuation)

Initialize some tools and create noise tokens list from punctuation

In [56]:
noise = list(punctuation)
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

Take a look on our questions

In [57]:
data_train

0        Как зовут лодочника на реке Стикс в древнегреч...
1                         Как в химии обозначается свинец?
2        Какой химический элемент преобладает в составе...
3        Кто из перечисленных был пажом во времена Екат...
4                          Когда началась 2 мировая война?
                               ...                        
29995    Как называется игра, местом действия которой я...
29996                 Как в опере называют ведущую певицу?
29997                 На какой планете сутки длиннее года?
29998    Сколько лет пролежал на печи известный богатыр...
29999    В каком году основали Московский Государственн...
Name: Question, Length: 30000, dtype: object

I've already lemmatized the sentences for you not to waste time. Here we load cached data

In [58]:
#load file

with open('train_lemmas.pkl', 'rb') as dummp:
  x_train_lemma=pickle.load(dummp) 
'''
x_train_lemma=[]
for text in data_train:
  doc=Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for token in doc.tokens:
    token.lemmatize(morph_vocab)
  lemmas=[_.lemma for _ in doc.tokens if _.text not in noise]
  #print(lemmas)
  x_train_lemma.append(lemmas)
'''

#load file

with open('test_lemmas.pkl', 'rb') as dummp:
  x_test_lemma=pickle.load(dummp) 
'''
x_test_lemma=[]
for text in data_test:
  doc=Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for token in doc.tokens:
    token.lemmatize(morph_vocab)
  lemmas=[_.lemma for _ in doc.tokens if _.text not in noise]
  #print(lemmas)
  x_test_lemma.append(lemmas)
'''

'\nx_test_lemma=[]\nfor text in data_test:\n  doc=Doc(text)\n  doc.segment(segmenter)\n  doc.tag_morph(morph_tagger)\n  for token in doc.tokens:\n    token.lemmatize(morph_vocab)\n  lemmas=[_.lemma for _ in doc.tokens if _.text not in noise]\n  #print(lemmas)\n  x_test_lemma.append(lemmas)\n'

If you use another data you need to lemmatize it. You can do it using the code above. To save lemmas use the code below.

In [59]:
'''with open('train_lemmas.pkl', 'wb') as lemms:
  pickle.dump(x_train_lemma, lemms)
with open('test_lemmas.pkl', 'wb') as lemms:
  pickle.dump(x_test_lemma, lemms)'''

"with open('train_lemmas.pkl', 'wb') as lemms:\n  pickle.dump(x_train_lemma, lemms)\nwith open('test_lemmas.pkl', 'wb') as lemms:\n  pickle.dump(x_test_lemma, lemms)"

List of lemmas back to string

In [60]:
xtk=np.array([' '.join(i) for i in x_train_lemma])
xttk=np.array([' '.join(i) for i in x_test_lemma])
x_test=xttk

Ok. Everybody is doing it. No comments. Ok. Splitting the data for calculating metrics on unviewed samples

In [61]:
x_train,x_val,y_train,y_val=model_selection.train_test_split(xtk, ans.Answer,test_size=0.2)

Transform arrays to pandas DataFrame as it requires CatBoost

In [62]:
train_d=pd.DataFrame(x_train, columns=['Question'])
val_d=pd.DataFrame(x_val, columns=['Question'])
test_d=pd.DataFrame(x_test, columns=['Question'])

Defining a function to build and train our model

In [63]:
def fit_catboost_on_Clever(X_train, X_test, y_train, y_test, catboost_params={}, verbose=100):
    learn_pool = Pool(
        X_train,
        y_train,
        text_features=['Question'],
        feature_names=list(X_train.columns),
    )
    test_pool = Pool(
        X_test,
        y_test,
        text_features=['Question'],
        feature_names=list(X_train.columns)
    )

    catboost_default_params = {
        'iterations': 500,
        'learning_rate': 0.1,
        'eval_metric': 'AUC',
        'task_type':'GPU'
    }

    catboost_default_params.update(catboost_params)

    model = CatBoostClassifier(**catboost_default_params)
    model.fit(learn_pool, eval_set=test_pool, verbose=verbose)

    return model



Loading the pretrained model not to waste time on fitting it again

In [64]:
model=catboost.CatBoostClassifier()
model.load_model(fname='CatBoost_Clever_by_ElijahKamski')

If you need to refit it just uncomment the string below

In [65]:
#model=fit_catboost_on_Clever(train_d,val_d, y_train,y_val)

Calculating our predictions and saving it to file

In [66]:
pd.DataFrame(model.predict_proba(test_d)[:,1], index=list(range(30001, 30000+test_d.shape[0]+1))).to_csv('answer_cat.csv', header=None)

If you for any reason need to save model again you know what to do

In [67]:
#model.save_model('CatBoost_Clever_by_ElijahKamski')

Thank you for interesting in my projects. And special thanks to the MIPT for this forum. I hope some time I would learn here and work maybe in Mail.Ru Group. Who knows what is going to happen?)